In [76]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPool2D,Dense,Dropout,BatchNormalization,Flatten,Input
from sklearn.model_selection import train_test_split


In [77]:
path = "UTKFace/UTKFace"
data = []
age = []
for img in os.listdir(path):
  ages = img.split("_")[0]
  img = cv2.imread(str(path)+"/"+str(img))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  data.append(np.array(img))
  age.append(np.array(ages))
age = np.array(age,dtype=np.int64)
data = np.array(data)


In [78]:
print(data.shape)
print(age.shape)

(23708, 200, 200, 3)
(23708,)


In [79]:
#split data
x_train,x_test,y_train,y_test = train_test_split(data,age,random_state=42)


In [80]:

input = Input(shape=(200,200,3))
conv1 = Conv2D(16,(2,2),activation="relu")(input)
pool1 = MaxPool2D((2,2))(conv1)
batch1 = BatchNormalization()(pool1)
Dropout1 = Dropout(0.5)(batch1)

conv2 = Conv2D(32,(2,2),activation="relu")(Dropout1)
pool2 = MaxPool2D((2,2))(conv2)
batch2 = BatchNormalization()(pool2)
Dropout2 = Dropout(0.5)(batch2)


conv3 = Conv2D(64,(2,2),activation="relu")(Dropout2)
pool3 = MaxPool2D((2,2))(conv3)
batch3 = BatchNormalization()(pool3)
Dropout3 = Dropout(0.5)(batch3)

flt = Flatten()(batch3)
age_l = Dense(64,activation="relu")(flt)
age_l = Dense(32,activation="relu")(age_l)
age_l = Dense(16,activation="relu")(age_l)
age_l = Dense(1,activation="sigmoid")(age_l)

model = Model(inputs=input,outputs= age_l)
model.summary()


Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 199, 199, 16)      208       
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 99, 99, 16)        0         
_________________________________________________________________
batch_normalization_60 (Batc (None, 99, 99, 16)        64        
_________________________________________________________________
dropout_49 (Dropout)         (None, 99, 99, 16)        0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 98, 98, 32)        2080      
_________________________________________________________________
max_pooling2d_59 (MaxPooling (None, 49, 49, 32)        0  

In [81]:
model.compile(optimizer="adam",loss="mse",metrics=['mae'])
model_CNN  = model.fit(x_train, y_train,
                        validation_split = 0.1,
                        batch_size = 128,
                        epochs=100)
model.save("Agemodel2.h5")

Epoch 1/100
126/126 [==============================] - 196s 2s/step - loss: 1421.7493 - mae: 32.0776 - val_loss: 1470.6379 - val_mae: 32.5447
Epoch 2/100
126/126 [==============================] - 193s 2s/step - loss: 1452.5786 - mae: 32.4330 - val_loss: 1470.6379 - val_mae: 32.5447
Epoch 3/100
126/126 [==============================] - 193s 2s/step - loss: 1437.0786 - mae: 32.1102 - val_loss: 1470.6379 - val_mae: 32.5447
Epoch 4/100
126/126 [==============================] - 193s 2s/step - loss: 1418.9166 - mae: 32.1555 - val_loss: 1470.6379 - val_mae: 32.5447
Epoch 5/100
126/126 [==============================] - 193s 2s/step - loss: 1417.2246 - mae: 32.1985 - val_loss: 1470.6379 - val_mae: 32.5447
Epoch 6/100
126/126 [==============================] - 194s 2s/step - loss: 1419.7277 - mae: 32.1628 - val_loss: 1470.6379 - val_mae: 32.5447
Epoch 7/100
126/126 [==============================] - 193s 2s/step - loss: 1443.1011 - mae: 32.3939 - val_loss: 1470.6379 - val_mae: 32.5447
Epoch 

In [86]:
# Test model
test_model = model.evaluate(x_test, y_test)

186/186 [==============================] - 23s 124ms/step - loss: 1447.1897 - mae: 32.4102


In [84]:
model.save("modelAgeregression2.h5")

In [87]:
from sklearn.metrics import mean_absolute_error
predicted = model.predict(x_test)
mse=mean_absolute_error(y_test, predicted)

32.41015691686405